In [25]:
import sys
sys.path.insert(0, yolov5_script_path)


In [18]:
import yaml
import os

# This script sets the CORRECT ABSOLUTE path in all data YAML files.

yaml_base_path = r"C:\Users\peter\Masters\Project\LUNA16\yolov5_output"
print(f"Searching for and correcting YAML files in subdirectories of: {yaml_base_path}")

# Loop through all 10 fold_data directories
for i in range(10):
    fold_dir = os.path.join(yaml_base_path, f"fold_{i}_data")
    if not os.path.isdir(fold_dir):
        print(f"Warning: Directory not found, skipping: {fold_dir}")
        continue

    # Find all luna16*.yaml files in the directory
    for file_name in os.listdir(fold_dir):
        if file_name.startswith("luna16") and file_name.endswith(".yaml"):
            yaml_path = os.path.join(fold_dir, file_name)

            try:
                with open(yaml_path, 'r') as f:
                    data = yaml.safe_load(f)

                data['path'] = fold_dir.replace('\\', '/')

                with open(yaml_path, 'w') as f:
                    yaml.dump(data, f, sort_keys=False, indent=2)

                print(f"Corrected path in: {yaml_path}")

            except Exception as e:
                print(f"Could not process {yaml_path}: {e}")

print("\nYAML path correction complete.")

Searching for and correcting YAML files in subdirectories of: C:\Users\peter\Masters\Project\LUNA16\yolov5_output
Corrected path in: C:\Users\peter\Masters\Project\LUNA16\yolov5_output\fold_0_data\luna16.yaml
Corrected path in: C:\Users\peter\Masters\Project\LUNA16\yolov5_output\fold_0_data\luna16_subset_10.yaml
Corrected path in: C:\Users\peter\Masters\Project\LUNA16\yolov5_output\fold_0_data\luna16_subset_25.yaml
Corrected path in: C:\Users\peter\Masters\Project\LUNA16\yolov5_output\fold_0_data\luna16_subset_50.yaml
Corrected path in: C:\Users\peter\Masters\Project\LUNA16\yolov5_output\fold_0_data\luna16_subset_75.yaml
Corrected path in: C:\Users\peter\Masters\Project\LUNA16\yolov5_output\fold_1_data\luna16.yaml
Corrected path in: C:\Users\peter\Masters\Project\LUNA16\yolov5_output\fold_1_data\luna16_subset_10.yaml
Corrected path in: C:\Users\peter\Masters\Project\LUNA16\yolov5_output\fold_1_data\luna16_subset_25.yaml
Corrected path in: C:\Users\peter\Masters\Project\LUNA16\yolov5_ou

In [9]:
import os
import subprocess
import re
import pandas as pd
import sys

experiments_to_run = [
    # 10% Data Experiments
    {'name': 'Custom_Sequential', 'data_p': 10, 'path': r"C:\Users\peter\Masters\Project\LUNA16\yolov5_output_custom_backbone_sequential_0.002lr_data_percentage_comparison_p10"},
    {'name': 'Custom_Full_Backbone', 'data_p': 10, 'path': r"C:\Users\peter\Masters\Project\LUNA16\yolov5_output_custom_full_backbone_0.002lr_data_percentage_comparison_p10"},
    {'name': 'Pretrained_COCO', 'data_p': 10, 'path': r"C:\Users\peter\Masters\Project\LUNA16\yolov5_output_pretrained_backbone_0.002lr_data_percentage_comparison_p10"},
    {'name': 'Random_Init', 'data_p': 10, 'path': r"C:\Users\peter\Masters\Project\LUNA16\yolov5_output_random_backbone_0.01lr_data_percentage_comparison_p10"},

    # 25% Data Experiments
    {'name': 'Custom_Sequential', 'data_p': 25, 'path': r"C:\Users\peter\Masters\Project\LUNA16\yolov5_output_custom_backbone_sequential_0.002lr_data_percentage_comparison_p25"},
    {'name': 'Custom_Full_Backbone', 'data_p': 25, 'path': r"C:\Users\peter\Masters\Project\LUNA16\yolov5_output_custom_full_backbone_0.002lr_data_percentage_comparison_p25"},
    {'name': 'Pretrained_COCO', 'data_p': 25, 'path': r"C:\Users\peter\Masters\Project\LUNA16\yolov5_output_pretrained_backbone_0.002lr_data_percentage_comparison_p25"},
    {'name': 'Random_Init', 'data_p': 25, 'path': r"C:\Users\peter\Masters\Project\LUNA16\yolov5_output_random_backbone_0.01lr_data_percentage_comparison_p25"},

    # 50% Data Experiments
    {'name': 'Custom_Sequential', 'data_p': 50, 'path': r"C:\Users\peter\Masters\Project\LUNA16\yolov5_output_custom_backbone_sequential_0.002lr_data_percentage_comparison_p50"},
    {'name': 'Custom_Full_Backbone', 'data_p': 50, 'path': r"C:\Users\peter\Masters\Project\LUNA16\yolov5_output_custom_full_backbone_0.002lr_data_percentage_comparison_p50"},
    {'name': 'Pretrained_COCO', 'data_p': 50, 'path': r"C:\Users\peter\Masters\Project\LUNA16\yolov5_output_pretrained_backbone_0.002lr_data_percentage_comparison_p50"},
    {'name': 'Random_Init', 'data_p': 50, 'path': r"C:\Users\peter\Masters\Project\LUNA16\yolov5_output_random_backbone_0.01lr_data_percentage_comparison_p50"},

    # 75% Data Experiments
    {'name': 'Custom_Sequential', 'data_p': 75, 'path': r"C:\Users\peter\Masters\Project\LUNA16\yolov5_output_custom_backbone_sequential_0.002lr_data_percentage_comparison_p75"},
    {'name': 'Custom_Full_Backbone', 'data_p': 75, 'path': r"C:\Users\peter\Masters\Project\LUNA16\yolov5_output_custom_full_backbone_0.002lr_data_percentage_comparison_p75"},
    {'name': 'Pretrained_COCO', 'data_p': 75, 'path': r"C:\Users\peter\Masters\Project\LUNA16\yolov5_output_pretrained_backbone_0.002lr_data_percentage_comparison_p75"},
    {'name': 'Random_Init', 'data_p':75, 'path': r"C:\Users\peter\Masters\Project\LUNA16\yolov5_output_random_backbone_0.01lr_data_percentage_comparison_p75"},

    # 100% Data Experiments
    {'name': 'Custom_Sequential', 'data_p': 100, 'path': r"C:\Users\peter\Masters\Project\LUNA16\yolov5_output_custom_backbone_sequential_0.002lr"},
    {'name': 'Custom_Full_Backbone', 'data_p': 100, 'path': r"C:\Users\peter\Masters\Project\LUNA16\yolov5_output_custom_full_backbone_0.002lr_p100"},
    {'name': 'Random_Init', 'data_p': 100, 'path': r"C:\Users\peter\Masters\Project\LUNA16\yolov5_output_random_init"},
    {'name': 'Pretrained_COCO', 'data_p': 100, 'path': r"C:\Users\peter\Masters\Project\LUNA16\yolov5_output"},
]

# Static Paths
yaml_base_path = r"C:\Users\peter\Masters\Project\LUNA16\yolov5_output"
yolov5_script_path = r"C:\Users\peter\Masters\Project\LUNA16\yolov5"

all_results = []

# Loop Through All Experiments
for run_info in reversed(experiments_to_run):
    model_name = run_info['name']
    data_p = run_info['data_p']
    base_path = run_info['path']

    for fold in range(10):
        # Dynamically build the path to the weights file
        if data_p < 100:
            weights_path = os.path.join(base_path, f"fold_{fold}_p{data_p}", f"luna16_nodule_detection_p{data_p}", "weights", "best.pt")
        else:
            weights_path = os.path.join(base_path, f"fold_{fold}", "luna16_nodule_detection", "weights", "best.pt")

        # Build path to the corresponding data.yaml file
        yaml_file_name = f"luna16_subset_{data_p}.yaml" if data_p < 100 else "luna16.yaml"
        data_yaml_path = os.path.join(yaml_base_path, f"fold_{fold}_data", yaml_file_name)

        print(weights_path)

        if not os.path.exists(weights_path) or not os.path.exists(data_yaml_path):
            continue


        # Get the absolute path to the val.py script
        val_script_path = os.path.join(yolov5_script_path, "val.py")

        command = [
            sys.executable,
            val_script_path,
            "--weights", weights_path,
            "--data", data_yaml_path,
            "--imgsz", "512",
            "--task", "val",
        ]

        print(f"Validating: {model_name} | {data_p}% Data | Fold {fold}")

        result = subprocess.run(command, capture_output=True, text=True, cwd=yolov5_script_path, shell=False)

        map_50 = 0.0
        map_50_95 = 0.0

        if result.returncode != 0:
            print(f"  -> ERROR: val.py failed for {model_name} p{data_p} fold {fold}")
        else:
            print(result)
            output_text = result.stderr
            match = re.search(r"all\s+\d+\s+\d+\s+[\d\.]+\s+[\d\.]+\s+([\d\.]+)\s+([\d\.]+)", output_text)
            if match:
                map_50 = float(match.group(1))
                map_50_95 = float(match.group(2))
            else:
                print(f"  -> WARNING: Could not parse mAP from successful run output for {model_name} p{data_p} fold {fold}")
                print("  -> Full output from val.py for this run:")
                print(output_text)

        all_results.append({
            "Model": model_name,
            "Data_Percentage": data_p,
            "Fold": fold,
            "mAP_50": map_50,
            "mAP_50_95": map_50_95
        })
        print(f"  -> mAP@.5: {map_50:.4f}, mAP@.5:.95: {map_50_95:.4f}")

# --- Save results to a CSV file ---
df = pd.DataFrame(all_results)
df.to_csv("validation_map_results_all_experiments_full_100.csv", index=False)

print("\nValidation complete! Results saved to validation_map_results_all_experiments.csv")

C:\Users\peter\Masters\Project\LUNA16\yolov5_output_custom_full_backbone_0.002lr_p100\fold_0\luna16_nodule_detection\weights\best.pt
Validating: Custom_Full_Backbone | 100% Data | Fold 0
CompletedProcess(args=['C:\\Users\\peter\\.conda\\envs\\DeepLearning\\python.exe', 'C:\\Users\\peter\\Masters\\Project\\LUNA16\\yolov5\\val.py', '--weights', 'C:\\Users\\peter\\Masters\\Project\\LUNA16\\yolov5_output_custom_full_backbone_0.002lr_p100\\fold_0\\luna16_nodule_detection\\weights\\best.pt', '--data', 'C:\\Users\\peter\\Masters\\Project\\LUNA16\\yolov5_output\\fold_0_data\\luna16.yaml', '--imgsz', '512', '--task', 'val'], returncode=0, stdout='', stderr="\x1b\x1bval: \x1bdata=C:\\Users\\peter\\Masters\\Project\\LUNA16\\yolov5_output\\fold_0_data\\luna16.yaml, weights=['C:\\\\Users\\\\peter\\\\Masters\\\\Project\\\\LUNA16\\\\yolov5_output_custom_full_backbone_0.002lr_p100\\\\fold_0\\\\luna16_nodule_detection\\\\weights\\\\best.pt'], batch_size=32, imgsz=512, conf_thres=0.001, iou_thres=0.6, m